In [2]:
import torch
import time

1. Multple views of storage

In [64]:
mat=torch.full((13, 13), 1) 
mat[:,1::5]=2
mat[1::6,:]=2
#mat[:,11]=2
mat[3:5,3:5]=3
mat[3:5,8:10]=3
mat[8:10,3:5]=3
mat[8:10,8:10]=3
mat

tensor([[1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.]])

2. Flops per second

In [114]:
mat1= torch.empty(5000, 5000)
mat2= torch.empty(5000, 5000)

In [115]:
mat1= mat1.normal_
mat2= mat2.normal_

In [120]:
begin=time.perf_counter()
multiplication=torch.mm(mat1(), mat2())
end=time.perf_counter()

In [121]:
multiplication

tensor([[  -4.5635,  -36.0604,   22.7584,  ...,   56.7895,   79.1862,
           53.8350],
        [-106.5351,  174.6922,   99.9550,  ...,  -24.9026,   37.6865,
          -24.1570],
        [  57.2366,  -48.3925,  -46.9093,  ..., -140.8684,   47.5566,
           16.4252],
        ...,
        [  71.3366, -158.1385,   62.5146,  ...,   50.6169,  -93.7723,
           64.7180],
        [  72.3728,  -83.9175,   12.4736,  ...,  -27.7741,   26.6830,
           77.0833],
        [  46.8185, -113.3306,   49.9084,  ..., -155.0277,  142.4183,
          111.4827]])

In [144]:
(multiplication.size()[0])**3/(end-begin)

133595752727.6793

torch.Size([5000, 5000])

3. playing with strides

In [150]:
def mul_row(tens):
    new_ten = torch.empty_like(tens)
    for i in range(tens.size()[0]):
        #print(tens[i,:])
        new_ten[i,:]=tens[i,:]*(i+1)
    return new_ten
        
    

In [192]:
m=torch.full((4,8),2.0)

In [193]:
mul_row(m)

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])

In [229]:
def mul_row_fast(tens):
    new_ten = torch.empty_like(tens)
    #m =tens.size()[0]
    n =tens.size()[0]
    multiplier = torch.arange(1,n+1).view(-1,1)
    new_ten = torch.mul(tens, multiplier)
    return new_ten
    

In [246]:
mul_row_fast(m)

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])

In [221]:
# def mul_row_fast(m):
#     d = m.size(0)
#     mat = torch.arange(1, d+1).view(-1,1).float()
#     return m.mul(mat)

In [236]:
mat3= torch.ones(1000, 400)
mat3
#mat2= torch.empty(1000, 5000)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [239]:
begin1=time.perf_counter()
rest=mul_row(mat3)
end1=time.perf_counter()

In [240]:
begin2=time.perf_counter()
restt = mul_row_fast(mat3)
end2=time.perf_counter()


In [245]:
rest==restt
end1-begin1, end2-begin2

(0.034802217000105884, 0.0035463269996398594)

In [4]:
w1 = torch.empty(50, 784).normal_(0,1e-6)

In [5]:
w1

tensor([[ 7.9413e-07, -5.0316e-07, -2.4404e-07,  ...,  5.1160e-07,
         -3.7250e-07,  1.3436e-06],
        [-3.4928e-07,  1.7651e-06, -1.9967e-06,  ...,  8.9738e-07,
         -9.1610e-07, -1.1091e-07],
        [ 1.1481e-06, -1.7357e-06,  9.8055e-07,  ...,  1.4667e-06,
          8.9313e-07,  2.4313e-06],
        ...,
        [-1.6421e-06, -1.1973e-06,  1.0879e-06,  ...,  6.3147e-07,
         -9.4178e-07, -1.1636e-06],
        [ 5.8699e-07,  9.8351e-07,  1.6214e-09,  ..., -8.1320e-07,
          3.7540e-08, -4.3785e-07],
        [ 1.4835e-06,  8.3148e-07,  9.9480e-07,  ..., -1.1980e-06,
         -2.6302e-06,  3.3405e-07]])